In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from tensorflow import device
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/snacks

In [ ]:
#!unzip foods_final.zip

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
batch_size = 20
epochs = 20
train_dir = 'train/'
test_dir = 'test/'

In [ ]:
# ImageDataGenerator used for training
# Decided to ultimately not use image augmentation due to long training times
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    validation_split=0.1,
    #rotation_range = 30,
    #zoom_range = 0.15,
    #width_shift_range = 0.1,
    #height_shift_range = 0.1,
    #shear_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest'
  )

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    shuffle = True,
    seed = 42,
    subset = "training"
)

valid_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    batch_size = batch_size,
    shuffle = True,
    seed = 42,
    subset = "validation"
)

In [ ]:
# MobileNetV2 to be finetuned for our data
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape = (IMG_HEIGHT, IMG_WIDTH, 3))
base_out = base_model.output
out = Dense(6, activation="sigmoid")(GlobalAveragePooling2D()(base_out))
model = Model(inputs = base_model.input, outputs = out)

In [ ]:
# Froze first 130 layers, finetuned the latter ones
for layer in model.layers[:130]:
    layer.trainable = False
for layer in model.layers[130:]:
    layer.trainable = True

model.compile(optimizer="adam",
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
with device('/device:GPU:0'):
  callbacks = [EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)]
  history = model.fit(train_generator,
                      validation_data = valid_generator,
                      epochs=epochs, callbacks=callbacks, verbose=1)

In [ ]:
model.save("image_model.keras")

In [ ]:
model = load_model('image_model.keras')

In [ ]:
# Testing against validation data
valid_generator.reset()
y_true = np.array([])
i = 0
while i < valid_generator.__len__():
  _, y = valid_generator.next()
  y_true = np.append(y_true, np.argmax(y, axis=1))
  i += 1

valid_generator.reset()
y_pred = model.predict(valid_generator)
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
# Validation confusion matrix
pd.DataFrame(
    confusion_matrix(y_true, y_pred),
    index=['true:cakes', 'true:candy', 'true:chips', 'true:chocolate', 'true:cookies', 'true:seeds'],
    columns=['pred:cakes', 'pred:candy', 'pred:chips', 'pred:chocolate', 'pred:cookies', 'pred:seeds'])

,pred:cakes,pred:candy,pred:chips,pred:chocolate,pred:cookies,pred:seeds
true:cakes,246,35,15,32,19,31
true:candy,14,494,37,74,9,130
true:chips,1,20,281,6,3,57
true:chocolate,10,69,12,227,11,48
true:cookies,51,38,22,49,276,92
true:seeds,5,29,43,21,6,660


In [ ]:
# Preprocessing function to predict on all of the data
target_size = (IMG_HEIGHT, IMG_WIDTH)
def preprocess_img(image_path, target_size=target_size):
  img = load_img(image_path, target_size=target_size)
  img_array = img_to_array(img)
  img_array = preprocess_input(img_array)
  img_array = np.expand_dims(img_array, axis=0)
  return img_array

In [ ]:
res_dict = {0:"cake", 1:"candy", 2:"savory",
            3:"chocolate", 4:"cookie", 5:"seeds"}
d = dict()
paths = ['train/cakes_cupcakes_snack_cakes', 'train/candy', 'train/chips_pretzels_snacks',
         'train/chocolate', 'train/cookies_biscuits', 'train/popcorn_peanuts_seeds_related_snacks',
         'test/unlabeled']

In [ ]:
# Predicting on all of the data - top 1 prediction
with device('/device:GPU:0'):
  for image_folder in paths:
    for filename in os.listdir(image_folder):
      if filename.endswith('.jpg'):
            image_path = os.path.join(image_folder, filename)
            processed_image = preprocess_img(image_path, target_size)
            prediction = model.predict(processed_image)
            predicted_value = np.argmax(prediction[0]) # Get prediction
            image_number = int(filename.split('.')[0])
            d[image_number] = res_dict[predicted_value]

In [ ]:
df = pd.DataFrame.from_dict(d, orient="index", columns=["img_class"])
df.to_csv("image_classification.csv")

In [ ]:
d = dict()

In [ ]:
# Predicting on all of the data - probabilities and ranks
with device('/device:GPU:0'):
  for image_folder in paths:
    for filename in os.listdir(image_folder):
      if filename.endswith('.jpg'):
            image_path = os.path.join(image_folder, filename)
            processed_image = preprocess_img(image_path, target_size)
            prediction = model.predict(processed_image)
            res = list(prediction[0]) + list(prediction[0].argsort().argsort()) # Get probabilities and their respective ranks
            image_number = int(filename.split('.')[0])
            d[image_number] = res

In [ ]:
df = pd.DataFrame.from_dict(d, orient="index", columns=["prob_" + cat for cat in ["cake", "candy", "savory", "chocolate", "cookie", "seeds"]]+\
                                                       ["rank_" + cat for cat in ["cake", "candy", "savory", "chocolate", "cookie", "seeds"]])
df.to_csv("image_data.csv")